In [2]:
with open('doc.txt',"r",encoding="utf-8") as f:
    raw_data=f.read()

In [3]:
import tiktoken
tokenizer=tiktoken.get_encoding("gpt2")

In [4]:
enc_text=tokenizer.encode(raw_data)
print(len(enc_text))

1570


In [7]:
context_length=4
x=enc_text[:context_length]
y=enc_text[context_length]
print(f"{x}--->{y}")

[8582, 234, 247, 383]--->6869


In [8]:
for i in range (1,context_length+1):
    context=enc_text[:i]
    desired=enc_text[i]
    print(tokenizer.decode(context),"--->",tokenizer.decode([desired]))
   



� ---> �
� ---> �
🌙 --->  The
🌙 The --->  Moon


In [9]:
import torch
from torch.utils.data import Dataset

class GPTdatasetv1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids = []

        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        for i in range(0, len(token_ids) - max_length, stride):
            input_chunks = token_ids[i : i + max_length]
            target_chunks = token_ids[i + 1 : i + max_length + 1]

            self.input_ids.append(torch.tensor(input_chunks, dtype=torch.long))
            self.target_ids.append(torch.tensor(target_chunks, dtype=torch.long))

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx], self.target_ids[idx]
    
 

/Users/owner/Desktop/GPT-2/venv/lib/python3.13/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [12]:
from torch.utils.data import DataLoader
import tiktoken

def create_dataloader_v1(
        txt,
        batch_size=4,
        max_length=256,
        stride=128,
        shuffle=True,
        drop_last=True,
        num_workers=0):

    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTdatasetv1(txt, tokenizer, max_length, stride)

    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )

In [13]:
import torch
dataloader=create_dataloader_v1(raw_data,batch_size=8,max_length=4,stride=1,shuffle=False)
data_iter=iter (dataloader)
inputs,target=next(data_iter)
print("INPUTS\n",inputs)
print("targets\n",target)



INPUTS
 tensor([[8582,  234,  247,  383],
        [ 234,  247,  383, 6869],
        [ 247,  383, 6869,   12],
        [ 383, 6869,   12,   50],
        [6869,   12,   50, 2287],
        [  12,   50, 2287,  276],
        [  50, 2287,  276, 6785],
        [2287,  276, 6785,  851]])
targets
 tensor([[ 234,  247,  383, 6869],
        [ 247,  383, 6869,   12],
        [ 383, 6869,   12,   50],
        [6869,   12,   50, 2287],
        [  12,   50, 2287,  276],
        [  50, 2287,  276, 6785],
        [2287,  276, 6785,  851],
        [ 276, 6785,  851,  317]])


>token embeddings


In [14]:
import torch
vocab_size=50257
output_dim=256
torch.manual_seed(123)  #random value alwase be constant when it is run 
embedding_layer=torch.nn.Embedding(vocab_size,output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.3035,  ...,  1.3337,  0.0771, -0.0522],
        [ 0.2386,  0.1411, -1.3354,  ..., -0.0315, -1.0640,  0.9417],
        [-1.3152, -0.0677, -0.1350,  ..., -0.3181, -1.3936,  0.5226],
        ...,
        [ 0.5871, -0.0572, -1.1628,  ..., -0.6887, -0.7364,  0.4479],
        [ 0.4438,  0.7411,  1.1263,  ...,  1.2091,  0.6781,  0.3331],
        [-0.2537,  0.1446,  0.7203,  ..., -0.2134,  0.2144,  0.3006]],
       requires_grad=True)


In [16]:
input_ids=torch.tensor([1,4,5,4,3,5])
print(embedding_layer(input_ids)) 


tensor([[ 0.2386,  0.1411, -1.3354,  ..., -0.0315, -1.0640,  0.9417],
        [ 0.5229,  0.1553,  0.5247,  ..., -0.2004,  0.8093, -0.6667],
        [-0.3162,  1.2700, -0.0903,  ..., -0.4098,  0.4978, -0.3721],
        [ 0.5229,  0.1553,  0.5247,  ..., -0.2004,  0.8093, -0.6667],
        [ 0.2579,  0.3420, -0.8168,  ..., -0.4840, -0.2713, -0.0774],
        [-0.3162,  1.2700, -0.0903,  ..., -0.4098,  0.4978, -0.3721]],
       grad_fn=<EmbeddingBackward0>)


In [17]:
max_length=4
dataloader=create_dataloader_v1(raw_data,batch_size=8,max_length=max_length,stride=1,shuffle=False)
data_iter=iter (dataloader)
inputs,target=next(data_iter)



In [18]:
print("INPUTS\n",inputs)
print("shape\n",inputs.shape)

INPUTS
 tensor([[8582,  234,  247,  383],
        [ 234,  247,  383, 6869],
        [ 247,  383, 6869,   12],
        [ 383, 6869,   12,   50],
        [6869,   12,   50, 2287],
        [  12,   50, 2287,  276],
        [  50, 2287,  276, 6785],
        [2287,  276, 6785,  851]])
shape
 torch.Size([8, 4])


In [19]:
token_embeddings=embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [23]:
context_length=max_length
pos_embedding_layer=torch.nn.Embedding(context_length,output_dim)
pos_embeddings=pos_embedding_layer(torch.arange(max_length))
print(pos_embeddings.shape)


torch.Size([4, 256])


In [24]:
input_embeddings = token_embeddings + pos_embeddings
print(input_embeddings.shape)

torch.Size([8, 4, 256])
